## KB Fuzzy Matching

In [218]:
import pandas as pd

In [63]:
import Levenshtein as lev

In [58]:
import rapidfuzz

In [59]:
### Match based on 

In [78]:
kb_entities=pd.read_csv('kb_entities.csv',names=['id','entity','desc'])

In [61]:
rapidfuzz.fuzz.ratio('Boris Johnson', 'Bojo')

35.29411764705882

In [70]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Bojo')

66.66666666666667

In [156]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Boris Kevin Jonshon')

69.56521739130434

In [157]:
rapidfuzz.fuzz.token_set_ratio('Boris Johnson', 'Boris Kevin Jonshon')

75.0

In [178]:
rapidfuzz.fuzz.token_set_ratio('Boris K Johnson','Boris Johnson')

76.47058823529412

In [159]:
rapidfuzz.fuzz.WRatio('Boris Johnson', 'Boris Kevin Jonshon')

75.0

In [160]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Bojo')

66.66666666666667

In [161]:
rapidfuzz.fuzz.token_set_ratio('Boris Johnson', 'Bojo')

47.05882352941177

In [162]:
rapidfuzz.fuzz.WRatio('Boris Johnson', 'Bojo')

60.00000000000001

In [68]:
lev.distance('Boris Johnson', 'Bojo')

10

In [198]:
rapidfuzz.fuzz.token_set_ratio('Borris Johnson', 'Dr. Boris K. D. Johnson PhD')

68.42105263157895

In [186]:
rapidfuzz.fuzz.ratio('boris johnson', 'boris')

55.55555555555556

In [72]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Boris')

100.0

In [69]:
lev.distance('Boris Johnson', 'Boris')

8

In [90]:
kb_entities[['id','entity']]

,id,entity
0,41210,John Burns
1,52782,Independent
2,37933,Sonia Sotomayor
3,36983,Samuel Alito
4,15108,Donald Trump
...,...,...
6751,243486,David Gordon
6752,407940,David Gordon
6753,412578,David Gordon
6754,41748,Palm


In [205]:
reference='Adam Smith'
distances={}
for entity_id in names.keys():
    distance=rapidfuzz.fuzz.token_set_ratio(reference,names[entity_id])
    distances[entity_id]=distance

In [206]:
K = 60
candidate_ids={key : val for key, val in distances.items() if val > K}
candidate_ids=list(candidate_ids.keys())

In [207]:
candidate_ids[0]

'53881'

In [208]:
kb_entities['id']=kb_entities['id'].astype(str)

In [209]:
candidate_df=kb_entities[kb_entities['id'].isin(candidate_ids)]

In [210]:
candidate_df.to_csv('candidates.csv')

In [127]:
### Match based on embedding similarity

In [174]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [175]:
names = dict()
descriptions = dict()
descriptions_vec=dict()

for row in candidate_df.iterrows():
    qid = row[1]['id']
    name = row[1]['entity']
    desc = row[1]['desc']
    names[qid] = name
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    descriptions[qid] = desc
    descriptions_vec[qid] = desc_enc

In [176]:
candidate_df[candidate_df['entity']=='Adam Smith']

,id,entity,desc
3415,13596,Adam Smith,US Representative from Washington (1997-presen...
3416,129552,Adam Smith,Moral philosopher and political economist
3417,256328,Adam Smith,San Diego realtor
3418,269916,Adam Smith,First National Bank of Coffee County
3419,379819,Adam Smith,Founder and CEO of Geoplex Australia


In [152]:
vector=descriptions_vec['13596']
distances={}
for entity_id in descriptions_vec.keys():
    vector_b=descriptions_vec[entity_id]
    distance=dot(vector, vector_b)/(norm(vector)*norm(vector_b))
    distances[entity_id]=distance

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in float_scalars
  """


In [153]:
res = dict(sorted(distances.items(), key = itemgetter(1), reverse = True)[:5])

In [154]:
res

{'13596': 0.99999994,
 '13573': 0.9508932,
 '13861': 0.9436378,
 '115495': 0.88238436,
 '200612': 0.88210636}

In [155]:
candidate_df[candidate_df['id'].isin(res.keys())]

,id,entity,desc
1104,13573,Adam Schiff,US Representative from California (2001-presen...
1600,115495,Eric Adams,\n Mayor of NYC; former Brooklyn Boroug...
1607,200612,Tim Smith,"President & CEO, Skidmore Studio Tim Smith, wh..."
1702,13861,Robert Smith,former US Representative and Senator from New ...
3415,13596,Adam Smith,US Representative from Washington (1997-presen...


In [199]:
candidates_alphabetical = {candidate.alias_ + ' ' + candidate.entity_: candidate for candidate in candidates}

candidates_alphabetical = dict(sorted(candidates_alphabetical.items(), key = itemgetter(0), reverse = True))

[candidate for candidate in candidates_alphabetical.values()]

In [159]:
'the Grateful Dead' in candidates_alphabetical.keys()

True

In [ ]:
import csv
from pathlib import Path

def load_entities():
    entities_loc = Path.cwd()/"kb_entities.csv"  # distributed alongside this notebook

    names = dict()
    descriptions = dict()
    descriptions_vec=dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            qid = row[0]
            name = row[1]
            desc = row[2]
            names[qid] = name
            desc_doc = nlp(desc)
            desc_enc = desc_doc.vector
            descriptions[qid] = desc
            descriptions_vec[qid] = desc_enc
    return names, descriptions, descriptions_vec

In [49]:
names, descriptions, descriptions_vec =load_entities()

In [ ]:
from numpy import dot
from numpy.linalg import norm

vector=descriptions['41210']
distances={}
for entity_id in descriptions.keys():
    vector_b=descriptions[entity_id]
    distance=dot(vector, vector_b)/(norm(vector)*norm(vector_b))
    distances[entity_id]=distance

In [36]:
from operator import itemgetter
res = dict(sorted(distances.items(), key = itemgetter(1), reverse = True)[:5])

In [10]:
import rapidfuzz

import spacy
from spacy.kb import KnowledgeBase, Candidate #, get_candidates

import prodigy
from prodigy.models.ner import EntityRecognizer
from prodigy.components.loaders import TXT
from prodigy.util import set_hashes
from prodigy.components.filters import filter_duplicates

In [11]:
nlp_dir='my_output/my_nlp'
kb_loc='my_output/my_kb'

In [25]:
nlp = spacy.load(nlp_dir)
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=1)
kb.from_disk(kb_loc)

In [18]:
span='Johnson'
aliases=kb.get_alias_strings()
match_scores={}
for al in aliases:
    fuzzy_ratio=rapidfuzz.fuzz.token_set_ratio(span.lower(),al.lower())
    if fuzzy_ratio >=99:
        match_scores[al]=fuzzy_ratio

candidates=[]
for match in match_scores.keys():
    candidates.extend(kb.get_alias_candidates(match))

In [69]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [123]:
ref_sentence='Boris Jonhson is a scoundrel politian who spent far too long on the UK Parliament.'

In [124]:
ref_sentence_spacy=nlp(ref_sentence).vector

In [125]:
len(ref_sentence_spacy)

300

In [128]:
aliases=kb.get_alias_strings()
candidates=[]
for al in aliases:
    candidates.extend(kb.get_alias_candidates(al))

In [130]:
len(candidates)

6756

In [63]:
names = dict()
descriptions_vec=dict()
for candidate in candidates:
    qid = candidate.entity_
    name = candidate.alias_
    desc_enc = candidate.entity_vector
    names[qid] = name
    descriptions_vec[qid] = desc_enc

In [77]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from operator import itemgetter

In [117]:
distances={}
for entity_id in descriptions_vec.keys():
    vector_b=descriptions_vec[entity_id]
    distance=np.nan_to_num(dot(ref_sentence_spacy.vector, vector_b)/(norm(ref_sentence_spacy.vector)*norm(vector_b)),0)
    distances[entity_id]=distance

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [118]:
res = dict(sorted(distances.items(), key = itemgetter(1), reverse = True)[:20])

In [119]:
vector_b=[candidate for candidate in selected_candidates if candidate.entity_=='231407'][0].entity_vector
dot(ref_sentence_spacy.vector, vector_b)/(norm(ref_sentence_spacy.vector)*norm(vector_b))

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


nan

In [120]:
res

{'277103': 0.8214952323983455,
 '67986': 0.8194300603879058,
 '46087': 0.8191962211405379,
 '247763': 0.8167178204327862,
 '414532': 0.7985223929282222,
 '159743': 0.7961500100947096,
 '161421': 0.7859962755192861,
 '175844': 0.7852419570087164,
 '192959': 0.7637444516616055,
 '247496': 0.761963452479024,
 '13365': 0.7599241701775898,
 '366432': 0.7587513473503269,
 '232981': 0.7515396883852269,
 '153744': 0.7514635891934693,
 '84062': 0.738965672530237,
 '409580': 0.7371468985349952,
 '340202': 0.7272839416030585,
 '91598': 0.726751742865994,
 '91597': 0.7240283513477187,
 '22635': 0.7184876190151583}

In [98]:
selected_candidates=[candidate for candidate in candidates if candidate.entity_ in res.keys()]

In [99]:
[selected_candidate.alias_ for selected_candidate in selected_candidates]

['Steve Johnson',
 'Andrew Johnson',
 'Tim Johnson',
 'Tim Johnson',
 'Lyndon B Johnson',
 'Dan Johnson',
 'Eric Johnson',
 'James Johnson',
 'Ron Johnson',
 'Paul Johnson',
 'Woody Johnson',
 'Harriet Johnson',
 'Mark Johnson',
 'Bill Johnson',
 'Bill Johnson',
 'Sam Johnson',
 'Boris Johnson',
 'Lee Johnson',
 'Dakota Johnson',
 'Alexis McGill Johnson']

In [155]:
candidate_df[candidate_df['id'].isin(res.keys())]

,id,entity,desc
1104,13573,Adam Schiff,US Representative from California (2001-presen...
1600,115495,Eric Adams,\n Mayor of NYC; former Brooklyn Boroug...
1607,200612,Tim Smith,"President & CEO, Skidmore Studio Tim Smith, wh..."
1702,13861,Robert Smith,former US Representative and Senator from New ...
3415,13596,Adam Smith,US Representative from Washington (1997-presen...


In [27]:
import spacy
from spacy.kb import KnowledgeBase, Candidate
import prodigy
from prodigy.models.ner import EntityRecognizer
import pandas as pd
import rapidfuzz
from operator import itemgetter

In [14]:
nlp_dir='my_output/nlp_lilsis'
kb_loc='my_output/kb_lilsis'
entity_loc='kb_datasets/kb_entities_lilsis.csv'

In [13]:
# Load the NLP and KB objects from file
nlp = spacy.load(nlp_dir)
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=1)
kb.from_disk(kb_loc)
model = EntityRecognizer(nlp)

In [17]:
# Read the pre-defined CSV file into dictionaries mapping QIDs to the full names and descriptions
id_dict = dict()

kb_entities=pd.read_csv(entity_loc, names=['qid','name','desc'])
for row in kb_entities.iterrows():
    qid = str(row[1][0])
    name = str(row[1][1])
    desc = str(row[1][2])
    id_dict[qid] = (name, desc)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
def get_candidates_from_fuzzy_matching(span, kb, matching_thres=60):
    """
    Return a list of candidate entities for an alias based on fuzzy string matching.
    Each candidate defines the entity, the original alias,
    and the prior probability of that alias resolving to that entity.
    If the alias is not known in the KB, and empty list is returned.
    """

    aliases=kb.get_alias_strings()
    #matches=[]
    matches={}
    for al in aliases:
        #fuzzy_ratio=rapidfuzz.fuzz.token_set_ratio(span.lower(),al.lower())
        fuzzy_ratio = rapidfuzz.fuzz.WRatio(span.lower(), al.lower())
        if fuzzy_ratio >=matching_thres:
            #matches.append(al)
            matches[al]=fuzzy_ratio

    candidates=[]
    for match in matches:
        candidates.extend(kb.get_alias_candidates(match))

    return candidates, matches

In [28]:
def order_candidates_fuzzy_score(candidates, matches, candidate_limit=10):
    """
    Order candidates by descending fuzzy name matching score
    """
    # names = dict()
    candidate_d = dict()
    fuzzy_scores = dict()
    for candidate in candidates:
        qid = candidate.entity_
        name = candidate.alias_
        # names[qid] = name
        candidate_d[qid] = candidate
        fuzzy_scores[qid] = matches[name]

    entities_ordered = dict(sorted(fuzzy_scores.items(), key=itemgetter(1), reverse=True))
    entities_ordered = list(entities_ordered.keys())[:candidate_limit]
    entities_ordered=[candidate_d[entity] for entity in entities_ordered]
    scores_ordered=
    return 

In [42]:
candidate_d = dict()
fuzzy_scores = dict()
for candidate in candidates:
    qid = candidate.entity_
    name = candidate.alias_
    # names[qid] = name
    candidate_d[qid] = candidate
    fuzzy_scores[qid] = matches[name]

entities_ordered = dict(sorted(fuzzy_scores.items(), key=itemgetter(1), reverse=True))
entities_ordered = list(entities_ordered.keys())[:10]


In [44]:
scores_ordered

[90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0]

In [22]:
candidates, matches = get_candidates_from_fuzzy_matching('Boris', kb)

In [29]:
candidates=order_candidates_fuzzy_score(candidates, matches)

In [ ]:
# Initialize the Prodigy stream by running the NER model
stream = TXT(source)
stream = [set_hashes(eg) for eg in stream]
stream = (eg for score, eg in model(stream))

# For each NER mention, add the candidates from the KB to the annotation task
stream = _add_options(stream, kb, nlp, id_dict)
stream = filter_duplicates(stream, by_input=False, by_task=True)

return {
    "dataset": dataset,
    "stream": stream,
    "view_id": "choice",
    "config": {"choice_auto_accept": False},
}

